<a href="https://colab.research.google.com/github/aswinaus/RAG/blob/main/RAG_with_Evaluation_using_RAGAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [ ]:
!pip install langchain langchain_community langchain_openai chromadb pymupdf nest_asyncio --quiet

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pymupdf

In [ ]:
# Download Data
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

In [ ]:
doc = pymupdf.open(f"{data_dir}/RAG/data/10k/lyft_10k_2023.pdf")

In [ ]:
#Printing the content to validate
for page in doc:
    text = page.get_text()
    print(text)

Streaming output truncated to the last 5000 lines.
financial measure. We encourage investors and others to review our financial information in its entirety, not to rely on any single financial measure and to view
our non-GAAP financial measures in conjunction with the respective most directly comparable GAAP financial measures.
67

Net loss is the most directly comparable financial measure to Adjusted EBITDA. The following table provides a reconciliation of net loss to Adjusted
EBITDA (in millions):
Year Ended December 31,
2023
2022
2021
(in millions)
Net loss
$
(340.3)
$
(1,584.5)
$
(1,062.1)
Adjusted to exclude the following:
Interest expense
29.7 
20.8 
52.8 
Other (income) expense, net
(170.1)
100.0 
(135.9)
Provision for (benefit from) income taxes
8.6 
5.9 
11.2 
Depreciation and amortization
116.5 
154.8 
139.3 
Stock-based compensation
484.5 
750.8 
724.6 
Payroll tax expense related to stock-based compensation
12.5 
17.0 
31.5 
Net amount from claims ceded under the Reinsuranc

In [ ]:
import chromadb
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
pages=[]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)
#You will need to run this one at a time for now.
loader = PyMuPDFLoader(f"{data_dir}/RAG/data/10k/lyft_10k_2023.pdf")
#loader = PyMuPDFLoader(f"{data_dir}/RAG/data/10k/uber_10k_2023.pdf")
# load_and_split uses RecursiveCharacterTextSplitter by default
pages_to_persist = loader.load_and_split(text_splitter)
pages.extend(pages_to_persist)

In [ ]:
# split the pages into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splits = text_splitter.split_documents(pages)

In [ ]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# create vector store with Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata # import filter_complex_metadata

vectordb = Chroma.from_documents(documents=pages, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]),persist_directory=f"{data_dir}/RAG/VectorDB/chroma_db_RAG")
vectordb.persist()
retriever = vectordb.as_retriever()

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
#Creating a RAG Pipeline
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# RAG
template = """You are an AI language model Accounting assistant.Answer the following question based on this context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])
final_rag_chain = (
    #{"context": retriever | format_docs, "question": RunnablePassthrough()}

    RunnablePassthrough.assign(
        context=lambda x: format_docs(vectordb.similarity_search(x["question"], k=10)),
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question="Can you compare the revenue for Uber and lyft for year 2023?"

In [ ]:
final_rag_chain.invoke({"question":question})

'Based on the information provided, the revenue for Uber in 2023 was $37,281 million. Unfortunately, there is no specific revenue information provided for Lyft in the given context, so a direct comparison of revenue for Uber and Lyft in 2023 cannot be made.'

In [ ]:
questions = [
    "Please explain the revolving credit of Lyft for year 2023 and its long term benefits?",
    "Could you give me a concise overview of the Business strategies employed by Uber and Lyft?",
    "What are the details of Lyft's revolving credit for the year 2023 and how does it contribute to long-term benefits?",
    "Could you analyze and compare the financial health of Uber and Lyft for the year 2023?",
    "When compared to Uber is Lyft financially stable?",
    "Can you compare the growth strategies between Uber and Lyft based on the 10K document for year 2023?",
    "Can you provide a summary of the Financial and Operational Highlights of Uber?"
    "How would you describe the Business overview of Uber and Lyft in a nutshell?",
    "How does Lyft's financial stability in 2023 compare to Uber's?",
    "Can you provide an overview of Lyft's revolving credit in 2023 and its potential long-term advantages?",
    "Can you please compare the financial status for both the companies for the year 2023?",
    "Can you compare the cost and expenses between Lyft and Uber for year 2023? Also let me know which company faired better?",
    "Can you provide a financial comparison between Uber and Lyft for the year 2023?",
    "Could you give me an overview of Lyft's Financial and Operational performance?",
    "In terms of financial stability, how does Lyft stack up against Uber for the year 2023?",
    "How does Lyft's revolving credit in 2023 work and what are the advantages it offers in the long run?",
    "When it comes to technology which one fairs better is it Uber or Lyft?",
    "What are the contrasting growth approaches of Uber and Lyft as outlined in the 2023 10K report?",
    "Could you give me a concise overview of the Business strategies employed by Uber and Lyft?",
    "Could you analyze and compare the financial health of Uber and Lyft for the year 2023?",
    "Can you summarize the important Financial and Operational metrics for Lyft",
    "When evaluating financial stability, how do Uber and Lyft differ in their performance for the year 2023?",
    ]
ground_truth = [
    "In 2023, Lyft entered into a revolving credit agreement with certain lenders for a 1.25 billion.  The long-term benefits of this revolving credit facility include providing Lyft with financial flexibility and access to capital when needed. It allows Lyft to borrow funds, repay them, and borrow again up to the agreed-upon limit, providing liquidity for operational needs, investments, or other strategic initiatives. Additionally, having a revolving credit facility in place can help Lyft manage its cash flow, take advantage of opportunities for growth, and navigate any unforeseen financial challenges.",
    "Based on the information provided in the 10K documents for 2023, both Uber and Lyft have growth strategies focused on increasing their user base, expanding their range of transportation services, and capturing a larger share of the transportation market. Lyft's growth strategy includes increasing rider use cases by offering various products and services such as Lyft Pink subscription plan, Lyft Pass commuter programs, and first-mile and last-mile services. They also aim to grow their share of consumers' transportation spend by providing a wide range of mobility options through their rideshare, bikes, and scooters network.On the other hand, Uber's growth strategy involves using their technology platform to connect consumers with various ride services, merchants, and delivery service providers. They also connect consumers with public transportation networks and provide solutions in the freight industry. Uber is also developing new technologies to solve everyday problems.Both companies face competition from other players in the market, and they are focused on innovation, technology, and expanding their services to stay competitive and attract more users.",
    "In 2023, Lyft entered into a revolving credit agreement with certain lenders for a 1.25 billion.  The long-term benefits of this revolving credit facility include providing Lyft with financial flexibility and access to capital when needed. It allows Lyft to borrow funds, repay them, and borrow again up to the agreed-upon limit, providing liquidity for operational needs, investments, or other strategic initiatives. Additionally, having a revolving credit facility in place can help Lyft manage its cash flow, take advantage of opportunities for growth, and navigate any unforeseen financial challenges.",
    "Financial Highlights: Lyft, Inc. operates multimodal transportation networks in the United States and Canada. The company's revenue is primarily generated from its ridesharing marketplace connecting drivers and riders. Lyft collects service fees and commissions from drivers for their use of the ridesharing marketplace. G ross Bookings and Rides increase as drivers accept more rider leads.Operational Highlights: Lyft offers an expanded set of transportation modes in select cities, including shared bikes and scooters for shorter rides and multimodal trips. The company's platform and mobile-based applications facilitate peer-to-peer ridesharing by connecting drivers with riders. Lyft aims to grow its share of consumers' transportation spend by providing a wide range of mobility options. The company focuses on delivering increasing value to drivers by offering economic opportunities and programs like Express Drive for access to rental cars.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "Based on the information provided in the 10K documents for 2023, both Uber and Lyft have growth strategies focused on increasing their user base, expanding their range of transportation services, and capturing a larger share of the transportation market. Lyft's growth strategy includes increasing rider use cases by offering various products and services such as Lyft Pink subscription plan, Lyft Pass commuter programs, and first-mile and last-mile services. They also aim to grow their share of consumers' transportation spend by providing a wide range of mobility options through their rideshare, bikes, and scooters network.On the other hand, Uber's growth strategy involves using their technology platform to connect consumers with various ride services, merchants, and delivery service providers. They also connect consumers with public transportation networks and provide solutions in the freight industry. Uber is also developing new technologies to solve everyday problems.Both companies face competition from other players in the market, and they are focused on innovation, technology, and expanding their services to stay competitive and attract more users.",
    "Financial Highlights: Lyft, Inc. operates multimodal transportation networks in the United States and Canada. The company's revenue is primarily generated from its ridesharing marketplace connecting drivers and riders. Lyft collects service fees and commissions from drivers for their use of the ridesharing marketplace. G ross Bookings and Rides increase as drivers accept more rider leads.Operational Highlights: Lyft offers an expanded set of transportation modes in select cities, including shared bikes and scooters for shorter rides and multimodal trips. The company's platform and mobile-based applications facilitate peer-to-peer ridesharing by connecting drivers with riders. Lyft aims to grow its share of consumers' transportation spend by providing a wide range of mobility options. The company focuses on delivering increasing value to drivers by offering economic opportunities and programs like Express Drive for access to rental cars.",
    "Lyft, Inc. started a movement to revolutionize transportation in 2012 with a peer-to-peer marketplace for on-demand ridesharing. They have continued to pioneer innovations and are now one of the largest multimodal transportation networks in the United States and Canada. Lyft's purpose is to get riders out into the world and provide drivers with control over their time and money. They offer a ridesharing marketplace through the Lyft App, connecting drivers with riders and providing various transportation modes. Uber Technologies, Inc. is a technology platform that powers movement from point A to point B. They connect consumers with providers of ride services, merchants, and delivery service providers. Uber also connects consumers with public transportation networks and carriers in the freight industry. They are developing technologies to provide new solutions for everyday problems.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "In 2023, Lyft entered into a revolving credit agreement with certain lenders for a 1.25 billion.  The long-term benefits of this revolving credit facility include providing Lyft with financial flexibility and access to capital when needed. It allows Lyft to borrow funds, repay them, and borrow again up to the agreed-upon limit, providing liquidity for operational needs, investments, or other strategic initiatives. Additionally, having a revolving credit facility in place can help Lyft manage its cash flow, take advantage of opportunities for growth, and navigate any unforeseen financial challenges.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "In 2023, Lyft reported total costs and expenses of 36,171 million. Comparing the two, Lyft had higher costs and expenses than Uber for the year 2023. Therefore, Uber fared better in terms of managing costs and expenses in 2023."
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "Financial Highlights: Lyft, Inc. operates multimodal transportation networks in the United States and Canada. The company's revenue is primarily generated from its ridesharing marketplace connecting drivers and riders. Lyft collects service fees and commissions from drivers for their use of the ridesharing marketplace. G ross Bookings and Rides increase as drivers accept more rider leads.Operational Highlights: Lyft offers an expanded set of transportation modes in select cities, including shared bikes and scooters for shorter rides and multimodal trips. The company's platform and mobile-based applications facilitate peer-to-peer ridesharing by connecting drivers with riders. Lyft aims to grow its share of consumers' transportation spend by providing a wide range of mobility options. The company focuses on delivering increasing value to drivers by offering economic opportunities and programs like Express Drive for access to rental cars.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "In 2023, Lyft entered into a revolving credit agreement with certain lenders for a 1.25 billion.  The long-term benefits of this revolving credit facility include providing Lyft with financial flexibility and access to capital when needed. It allows Lyft to borrow funds, repay them, and borrow again up to the agreed-upon limit, providing liquidity for operational needs, investments, or other strategic initiatives. Additionally, having a revolving credit facility in place can help Lyft manage its cash flow, take advantage of opportunities for growth, and navigate any unforeseen financial challenges.",
    "Based on the provided context, both Uber and Lyft heavily rely on technology to power their transportation services. Uber is described as a technology platform that uses leading technology to connect consumers with ride services, delivery services, and public transportation networks. Lyft, on the other hand, leverages its technology platform to connect drivers with riders through its ridesharing marketplace and offers an expanded set of transportation modes. In terms of technology, both Uber and Lyft have their strengths and focus areas. Uber emphasizes its technology applications supporting a variety of offerings on its platform, while Lyft highlights its robust technology platform that powers rides and connections every day. Ultimately, the effectiveness and success of each company's technology may vary based on specific features, innovations, and user experiences.",
    "Based on the information provided in the 10K documents for 2023, both Uber and Lyft have growth strategies focused on increasing their user base, expanding their range of transportation services, and capturing a larger share of the transportation market. Lyft's growth strategy includes increasing rider use cases by offering various products and services such as Lyft Pink subscription plan, Lyft Pass commuter programs, and first-mile and last-mile services. They also aim to grow their share of consumers' transportation spend by providing a wide range of mobility options through their rideshare, bikes, and scooters network.On the other hand, Uber's growth strategy involves using their technology platform to connect consumers with various ride services, merchants, and delivery service providers. They also connect consumers with public transportation networks and provide solutions in the freight industry. Uber is also developing new technologies to solve everyday problems.Both companies face competition from other players in the market, and they are focused on innovation, technology, and expanding their services to stay competitive and attract more users.",
    "Lyft, Inc. started a movement to revolutionize transportation in 2012 with a peer-to-peer marketplace for on-demand ridesharing. They have continued to pioneer innovations and are now one of the largest multimodal transportation networks in the United States and Canada. Lyft's purpose is to get riders out into the world and provide drivers with control over their time and money. They offer a ridesharing marketplace through the Lyft App, connecting drivers with riders and providing various transportation modes.Uber Technologies, Inc. is a technology platform that powers movement from point A to point B. They connect consumers with providers of ride services, merchants, and delivery service providers. Uber also connects consumers with public transportation networks and carriers in the freight industry. They are developing technologies to provide new solutions for everyday problems.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "Financial Highlights: Lyft, Inc. operates multimodal transportation networks in the United States and Canada. The company's revenue is primarily generated from its ridesharing marketplace connecting drivers and riders. Lyft collects service fees and commissions from drivers for their use of the ridesharing marketplace. Gross Bookings and Rides increase as drivers accept more rider leads.Operational Highlights: Lyft offers an expanded set of transportation modes in select cities, including shared bikes and scooters for shorter rides and multimodal trips. The company's platform and mobile-based applications facilitate peer-to-peer ridesharing by connecting drivers with riders. Lyft aims to grow its share of consumers' transportation spend by providing a wide range of mobility options. The company focuses on delivering increasing value to drivers by offering economic opportunities and programs like Express Drive for access to rental cars.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023."
    ]

In [ ]:
!pip install datasets --quiet
from datasets import Dataset

In [ ]:
answers  = []
contexts = []

# traversing each question and passing into the chain to get answer from the system
for question in questions:
    answers.append(final_rag_chain.invoke({"question":question}))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])

# Preparing the dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

dataset.to_pandas()

,question,answer,contexts,ground_truth
0,Please explain the revolving credit of Lyft fo...,The revolving credit facility of Lyft for the ...,[under the revolving credit facility. As of De...,"In 2023, Lyft entered into a revolving credit ..."
1,Could you give me a concise overview of the Bu...,Both Uber and Lyft are technology platforms th...,"[PART I\nItem 1. Business.\nOverview\nLyft, In...",Based on the information provided in the 10K d...
2,What are the details of Lyft's revolving credi...,"In 2023, Lyft, Inc. entered into a revolving c...",[under the revolving credit facility. As of De...,"In 2023, Lyft entered into a revolving credit ..."
3,Could you analyze and compare the financial he...,To analyze and compare the financial health of...,[— %\n— %\nNet income (loss) including non-con...,"Financial Highlights: Lyft, Inc. operates mult..."
4,When compared to Uber is Lyft financially stable?,"Based on the information provided, Lyft appear...",[from technological innovation in mobility.\nO...,"Based on the provided context, it appears that..."
5,Can you compare the growth strategies between ...,Based on the information provided in the 10K d...,[also provide centralized tools and enterprise...,Based on the information provided in the 10K d...
6,Can you provide a summary of the Financial and...,Financial and Operational Highlights of Uber:\...,"[UBER TECHNOLOGIES, INC.\nNOTES TO CONSOLIDATE...","Financial Highlights: Lyft, Inc. operates mult..."
7,How does Lyft's financial stability in 2023 co...,"Based on the information provided, Lyft's fina...",[from technological innovation in mobility.\nO...,"Lyft, Inc. started a movement to revolutionize..."
8,Can you provide an overview of Lyft's revolvin...,"In 2023, Lyft, Inc. entered into a revolving c...",[under the revolving credit facility. As of De...,"Based on the provided context, it appears that..."
9,Can you please compare the financial status fo...,"Based on the information provided, it appears ...",[Item 7. Management’s Discussion and Analysis ...,"In 2023, Lyft entered into a revolving credit ..."


In [ ]:
!pip install ragas --quiet
import ragas

In [ ]:
#!git clone https://github.com/aswinaus/rag_dataset_ragas.git
#%cd rag_dataset_ragas

In [ ]:
#from datasets import load_dataset
#dataset = load_dataset('json', data_files='RAGDataset.json')
#dataset = dataset['train']
#print(dataset)

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()
df

Evaluating:   0%|          | 0/84 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[32]: ValidationError(2 validation errors for Verification
reason
  Field required [type=missing, input_value={'text': '{"reason": "The...nswer.", "verdict": 0}'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
verdict
  Field required [type=missing, input_value={'text': '{"reason": "The...nswer.", "verdict": 0}'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing)


,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,Please explain the revolving credit of Lyft fo...,[under the revolving credit facility. As of De...,The revolving credit facility of Lyft for the ...,"In 2023, Lyft entered into a revolving credit ...",1.000000,0.000000,0.600000,0.953317
1,Could you give me a concise overview of the Bu...,"[PART I\nItem 1. Business.\nOverview\nLyft, In...",Both Uber and Lyft are technology platforms th...,Based on the information provided in the 10K d...,0.416667,0.571429,0.850000,0.907065
2,What are the details of Lyft's revolving credi...,[under the revolving credit facility. As of De...,"In 2023, Lyft, Inc. entered into a revolving c...","In 2023, Lyft entered into a revolving credit ...",1.000000,0.000000,0.772727,0.888783
3,Could you analyze and compare the financial he...,[— %\n— %\nNet income (loss) including non-con...,To analyze and compare the financial health of...,"Financial Highlights: Lyft, Inc. operates mult...",0.000000,0.000000,1.000000,0.973196
4,When compared to Uber is Lyft financially stable?,[from technological innovation in mobility.\nO...,"Based on the information provided, Lyft appear...","Based on the provided context, it appears that...",0.000000,0.000000,0.562500,0.000000
5,Can you compare the growth strategies between ...,[also provide centralized tools and enterprise...,Based on the information provided in the 10K d...,Based on the information provided in the 10K d...,0.000000,0.285714,0.214286,0.936991
6,Can you provide a summary of the Financial and...,"[UBER TECHNOLOGIES, INC.\nNOTES TO CONSOLIDATE...",Financial and Operational Highlights of Uber:\...,"Financial Highlights: Lyft, Inc. operates mult...",0.000000,0.500000,0.633333,0.942954
7,How does Lyft's financial stability in 2023 co...,[from technological innovation in mobility.\nO...,"Based on the information provided, Lyft's fina...","Lyft, Inc. started a movement to revolutionize...",0.000000,0.500000,1.000000,0.000000
8,Can you provide an overview of Lyft's revolvin...,[under the revolving credit facility. As of De...,"In 2023, Lyft, Inc. entered into a revolving c...","Based on the provided context, it appears that...",NaN,0.000000,0.647059,0.946010
9,Can you please compare the financial status fo...,[Item 7. Management’s Discussion and Analysis ...,"Based on the information provided, it appears ...","In 2023, Lyft entered into a revolving credit ...",0.000000,0.000000,0.285714,0.935744
